<a href="https://colab.research.google.com/github/thilaniShalinga/FYP/blob/TimeSeriesAnalysis_FBPropet/FbPropet_timeSeriesUpdated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kagglehub


In [ ]:
from google.colab import files
files.upload()  # upload kaggle.json


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"shalingaathilani","key":"e72dac2406485250f6d5afac68ac15a6"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("prasoonkottarathil/btcinusd")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/btcinusd


In [10]:
import pandas as pd

# Load the CSV file (update filename if different)
df = pd.read_csv("/kaggle/input/btcinusd/BTC-2021min.csv")  # or .csv filename inside that folder

# Show the first few rows
df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1646106180,2022-03-01 03:43:00,BTC/USD,43046.58,43046.58,43046.58,43046.58,0.000000,0.000000
1,1646106060,2022-03-01 03:41:00,BTC/USD,43018.23,43046.59,43018.23,43046.58,0.142977,6154.673021
2,1646106000,2022-03-01 03:40:00,BTC/USD,43022.24,43022.24,43016.03,43016.03,0.009230,397.037957
3,1646105940,2022-03-01 03:39:00,BTC/USD,43035.16,43035.16,42999.44,42999.44,0.820950,35300.390268
4,1646105880,2022-03-01 03:38:00,BTC/USD,43077.82,43077.82,43049.46,43049.46,0.022210,956.143143


In [11]:
import numpy as np
from prophet import Prophet
import holidays
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [12]:
df = df.rename(columns={'date': 'ds', 'close': 'y'})
df.head()

,unix,ds,symbol,open,high,low,y,Volume BTC,Volume USD
0,1646106180,2022-03-01 03:43:00,BTC/USD,43046.58,43046.58,43046.58,43046.58,0.000000,0.000000
1,1646106060,2022-03-01 03:41:00,BTC/USD,43018.23,43046.59,43018.23,43046.58,0.142977,6154.673021
2,1646106000,2022-03-01 03:40:00,BTC/USD,43022.24,43022.24,43016.03,43016.03,0.009230,397.037957
3,1646105940,2022-03-01 03:39:00,BTC/USD,43035.16,43035.16,42999.44,42999.44,0.820950,35300.390268
4,1646105880,2022-03-01 03:38:00,BTC/USD,43077.82,43077.82,43049.46,43049.46,0.022210,956.143143


In [13]:
df['ds'] = pd.to_datetime(df['ds']) #Convert the date column to datetime format

In [14]:
df = df.dropna() #Drop missing values if any

In [15]:
print(df.columns)
print(df.head())

Index(['unix', 'ds', 'symbol', 'open', 'high', 'low', 'y', 'Volume BTC',
       'Volume USD'],
      dtype='object')
         unix                  ds   symbol      open      high       low  \
0  1646106180 2022-03-01 03:43:00  BTC/USD  43046.58  43046.58  43046.58   
1  1646106060 2022-03-01 03:41:00  BTC/USD  43018.23  43046.59  43018.23   
2  1646106000 2022-03-01 03:40:00  BTC/USD  43022.24  43022.24  43016.03   
3  1646105940 2022-03-01 03:39:00  BTC/USD  43035.16  43035.16  42999.44   
4  1646105880 2022-03-01 03:38:00  BTC/USD  43077.82  43077.82  43049.46   

          y  Volume BTC    Volume USD  
0  43046.58    0.000000      0.000000  
1  43046.58    0.142977   6154.673021  
2  43016.03    0.009230    397.037957  
3  42999.44    0.820950  35300.390268  
4  43049.46    0.022210    956.143143  


In [28]:
from datetime import datetime

# Combine international and Sri Lankan holidays
intl_holidays = holidays.US(years=range(2015, 2026))  # Replace 'US' with other countries if needed
sri_lanka_holidays = holidays.SriLanka(years=range(2015, 2026))

combined_holidays = pd.DataFrame([
    {"ds": date, "holiday": "holiday"}
    for date in set(intl_holidays.keys()).union(sri_lanka_holidays.keys())
])

print(combined_holidays)

             ds  holiday
0    2023-12-26  holiday
1    2018-01-01  holiday
2    2023-11-12  holiday
3    2021-05-14  holiday
4    2020-12-25  holiday
..          ...      ...
391  2023-04-14  holiday
392  2016-11-11  holiday
393  2023-12-25  holiday
394  2023-04-05  holiday
395  2025-01-13  holiday

[396 rows x 2 columns]


In [26]:
# Use 90% for training and 10% for testing
train_size = int(len(df) * 0.9)
train_df = df[:train_size]
test_df = df[train_size:]

print(f"Training data size: {len(train_df)}")
print(f"Testing data size: {len(test_df)}")

Training data size: 549703
Testing data size: 61079


In [29]:
#Build Prophet Model with Holiday Integration
from prophet import Prophet

model = Prophet(
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=True,
    holidays=holidays  # this includes Sundays + other SL/international holidays
)
model.fit(train_df)


ValueError: holidays must be a DataFrame with "ds" and "holiday" columns.